# Tarea 1 <br/> CC6204 Deep Learning, Universidad de Chile  <br/> Hoja de respuestas partes 1 y 2 
## Nombre: Martin Cornejo Saavedra
Fecha sugerida para completar esta parte: 23 de marzo de 2018

In [0]:
# instalacion de los paquetes necesarios

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q ipdb

# para manejar archivos de Drive

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Nos debemos autentificar como usuarios
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import glob
import torch
import pdb

# Parte 1: Funciones de activación, derivadas y función de salida

## 1a) Funciones de activación

In [0]:
def relu(T):
    T[T < 0] = 0
    return T

def sig(T):
    return torch.reciprocal(1 + torch.exp(-1 * T))

def swish(T, beta):
    return torch.mul(T, sig(torch.mul(T, beta)))

def celu(T, alfa):
    positive = relu(T)
    negative = torch.mul(relu(torch.mul(T, -1)), -1)
    celu_T = torch.mul(torch.add(torch.exp(torch.div(negative, alfa)), -1), alfa)

    return torch.add(positive, 1, celu_T)

## 1b) Derivando las funciones de activación


\begin{equation}
\frac{\partial\ \text{relu}(x)}{\partial x} =
\left\{
	\begin{array}{ll}
		1  & \mbox{si } x \geq 0 \\
		0  & \mbox{~} 
	\end{array}
\right. 
\end{equation}
<br>

Dado $ \sigma (x) = sigmoid(x)$, tenemos que:

\begin{eqnarray}
\frac{\partial\ \text{swish}(x, \beta)}{\partial x} & = \sigma (\beta x) + \beta x \cdot \sigma (\beta x)(1-\sigma (\beta x)) \\
& = \sigma (\beta x) + \beta x \cdot \sigma (\beta x) - \beta x \cdot \sigma (\beta x)^{2}  \\
&= \beta \cdot swish(x, \beta) + \sigma (\beta x)(1 - \beta \cdot swish(x, \beta))\\
\\
\frac{\partial\ \text{swish}(x, \beta)}{\partial \beta} & =  
x^2 \sigma (\beta x)(1 - \sigma (\beta x))\\
\end{eqnarray}
<br><br>

\begin{eqnarray}
\frac{\partial\ \text{celu}(x, \alpha)}{\partial x} & =  
\left\{
	\begin{array}{ll}
		1  & \mbox{si } x \geq 0 \\
		exp (\frac{x}{\alpha})  & \mbox{~} 
	\end{array}
\right. \\
\\
\frac{\partial\ \text{celu}(x, \alpha)}{\partial \alpha} & = 
\left\{
	\begin{array}{ll}
		0  & \mbox{si } x \geq 0 \\
		exp (\frac{x}{\alpha})(1 - \frac{x}{\alpha}) - 1  & \mbox{~} 
	\end{array}
\right. \\
\end{eqnarray}

## 1c) Softmax

Dada la funcion `softmax` sabemos que cada elemento de la secuencia $\text{softmax}(x_1,\ldots,x_n)$ tiene la forma

\begin{equation}
s_i = \frac{e^{x_i}}{\sum_{j=1}^{n}e^{x_j}}
\end{equation}

Luego, para cada elemento de la secuencia $\text{softmax}(x_1-M,\ldots,x_n-M)$ se tiene

\begin{equation}
s_i = \frac{e^{x_i-M}}{\sum_{j=1}^{n}e^{x_j-M}} = \frac{e^{-M}e^{x_i}}{\sum_{j=1}^{n}e^{-M}e^{x_j}} = \frac{e^{-M}e^{x_i}}{e^{-M}\sum_{j=1}^{n}e^{x_j}} = \frac{e^{x_i}}{\sum_{j=1}^{n}e^{x_j}}
\end{equation}

Demostrando que $\text{softmax}(x_1-M,\ldots,x_n-M) = \text{softmax}(x_1,\ldots,x_n)$.

In [0]:
# por ahora softmax estara implementada solo para tensores en 2-D
def softmax(T, dim=0, estable=True):
    denom_softmax = torch.div(T, 2)
    denom_softmax = torch.exp(denom_softmax)
    denom_softmax = torch.mm(denom_softmax, torch.transpose(denom_softmax, 0, 1))
    denom_softmax = torch.reciprocal(torch.diag(denom_softmax))

    return torch.mm(torch.diag(denom_softmax), T.exp())

# Parte 2: Red neuronal y pasada hacia adelante (forward)

## 2a) Clase para red neuronal, 2b) Usando la GPU, 2c) Pasada hacia adelante

In [0]:
class FFNN():
    def __init__(self, F, l_h, l_a, C):
        self.F = F
        self.l_h = l_h
        self.l_a = l_a
        self.C = C

        self.W_1 = torch.randn(F, l_h[0])
        self.b_1 = torch.zeros(1, l_h[0])

        self.W_2 = torch.randn(l_h[0], l_h[1])
        self.b_2 = torch.zeros(1, l_h[1])

        self.U = torch.randn(l_h[1], C)
        self.c_init = torch.zeros(1, C)
  
    def gpu(self):
        if torch.cuda.is_available():
            self.W_1 = self.W_1.cuda()
            self.b_1 = self.b_1.cuda()
            self.W_2 = self.W_2.cuda()
            self.b_2 = self.b_2.cuda()
            self.U = self.U.cuda()
            self.c_init = self.c_init.cuda()
  
    def cpu(self):
        self.W_1 = self.W_1.cpu()
        self.b_1 = self.b_1.cpu()
        self.W_2 = self.W_2.cpu()
        self.b_2 = self.b_2.cpu()
        self.U = self.U.cpu()
        self.c_init = self.c_init.cpu()
  
    def forward(self, x):
        if torch.cuda.is_available():
          x = x.cuda()
          self.gpu()   # redundante, corregir
          
        h_1 = sig(torch.mm(x, self.W_1) + self.b_1)
        h_2 = sig(torch.mm(h_1, self.W_2) + self.b_2)
        y = softmax(torch.mm(h_2, self.U) + self.c_init)
        pdb.set_trace()

        return y

## 2d) Probando tu red con un modelo pre-entrenado

In [39]:
# cargar modelo desde Drive

local_download_path = os.path.expanduser('~/data_drive')
try:
  os.makedirs(local_download_path)
except: pass

folder_id = "1kuqGFqjqfDXcppXrRmPHCVUJzE7LRS1a"

file_list = drive.ListFile(
    {'q': "'{}' in parents".format(folder_id)}).GetList()

for f in file_list:
  # Creamos los archivos y los descargamos
  # pdb.set_trace()
  mimetypes = {
      # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

      # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
      
      # Drive folders
        'application/vnd.google-apps.folder': 'N/A'

      # see https://developers.google.com/drive/v3/web/mime-types
  }
  download_mimetype = None
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  #f_ = drive.CreateFile({'id': f['id']})
  #f_.GetContentFile(fname)
  
  # la carpeta tambien aparecen en la lista, pero no la queremos descargar
  # por eso cuando encontramos el tipo vnd.google-apps.folder lo saltamos
  if f['mimeType'] in mimetypes:
    continue
  else:
    f.GetContentFile(fname)

title: parametros_entrenados, id: 11qtO6OniHzVw5r5u_NAXbZP_dmNBQeL_
downloading to /content/data_drive/parametros_entrenados
title: W1.txt, id: 1WWPWOOG2tG3ju0dMkwq_ef6d3Lnm-W6F
downloading to /content/data_drive/W1.txt
title: U.txt, id: 1Jy8gbxmcxnAMQ-5kBTq044oR6tyZ_3Px
downloading to /content/data_drive/U.txt
title: b1.txt, id: 1KElNyYOVpaxw-mjegytrjg0RYUevDqjD
downloading to /content/data_drive/b1.txt
title: W2.txt, id: 1U8MqeA47K5Xy-HGiWggJuv7eiv3F5Hm8
downloading to /content/data_drive/W2.txt
title: b2.txt, id: 1eVE0NIfDVrFixBXOIUDuevd6Lp-887QF
downloading to /content/data_drive/b2.txt
title: c.txt, id: 1JfuB_nyAFvvI_VhV5e_rQ3Lm_rJdmbxx
downloading to /content/data_drive/c.txt


In [0]:
# cargar parametros entrenados a tensores
import glob
import numpy
params = []

for data_path in glob.glob(local_download_path + "/*.txt"):
    params.append(numpy.loadtxt(data_path))


In [50]:
os.chdir("/content/data_drive")
#!rm -R /content/Deep_learning/
!ls

#red_neuronal = FFNN(4096, [15,15], ['algo'], 10)
#red_neuronal.forward(torch.randn(3,4096))



b1.txt	c.txt						 U.txt	 W2.txt
b2.txt	Respuestas_Tarea_1_Partes_1_2_CC6402_2018.ipynb  W1.txt


In [0]:
# Tu código visualizando los ejemplos incorrectos acá